In [1]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, SeparableConv2D, BatchNormalization,
                                     Activation, MaxPooling2D, GlobalAveragePooling2D, Add,
                                    Dense, ReLU, Dropout, Concatenate)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import layers, models
import os

from sklearn.utils import class_weight
import numpy as np



2025-04-16 15:18:08.840898: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 15:18:08.847295: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-16 15:18:08.865047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 15:18:08.892292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 15:18:08.900112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 15:18:08.921377: I tensorflow/core/platform/cpu_feature_gu

In [25]:
# 1. Data setup
img_size = 64
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=10,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'emotions-binary/train',
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    'emotions-binary/test',
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary'
)

num_classes = len(train_data.class_indices)

Found 1521 images belonging to 2 classes.
Found 747 images belonging to 2 classes.


In [26]:
def build_mobilenetlike_model(input_shape=(64, 64, 1)):
    inputs = Input(shape=input_shape)

    x = SeparableConv2D(32, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(160, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(320, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    
    outputs = Dense(1, activation='sigmoid')(x)  # Binary output

    model = Model(inputs, outputs)
    return model

In [27]:
def small_res_block(x, filters):
    shortcut = x

    if x.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = SeparableConv2D(filters, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = SeparableConv2D(filters, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

In [28]:
def build_tiny_resnet(input_shape=(64, 64, 1)):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, 3, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = small_res_block(x, 32)
    x = MaxPooling2D()(x)  # Downsample

    x = small_res_block(x, 64)
    x = MaxPooling2D()(x)

    x = small_res_block(x, 128)
    x = GlobalAveragePooling2D()(x)

    outputs = Dense(1, activation='sigmoid')(x)  # Binary output

    model = Model(inputs, outputs)
    return model

In [29]:
def make_shallow_wide_model(input_shape=(64, 64, 1)):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)  # Binary output

    return models.Model(inputs, x)


In [30]:
def fire_module(x, squeeze_channels, expand_channels):
    squeeze = Conv2D(squeeze_channels, (1,1), activation='relu', padding='same')(x)

    expand1x1 = Conv2D(expand_channels, (1,1), activation='relu', padding='same')(squeeze)
    expand3x3 = Conv2D(expand_channels, (3,3), activation='relu', padding='same')(squeeze)

    output = Concatenate(axis=-1)([expand1x1, expand3x3])
    return output

In [31]:
def build_squeezenet(input_shape=(64, 64, 1)):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=2, padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=16, expand_channels=32)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=24, expand_channels=64)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=32, expand_channels=128)
    x = GlobalAveragePooling2D()(x)

    outputs = Dense(1, activation='sigmoid')(x)

    return Model(inputs, outputs)

In [32]:
model = build_mobilenetlike_model()
# model = build_tiny_resnet()
# model = make_shallow_wide_model()
# model = build_squeezenet()

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_10             │ (None, 64, 64, 32)     │            73 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_11             │ (None, 32, 32, 64)     │         2,400 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_12             │ (None, 16, 16, 160)    │        10,976 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 16, 16, 160)    │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 16, 16, 160)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 8, 8, 160)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_13             │ (None, 8, 8, 320)      │        52,960 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 8, 8, 320)      │         1,280 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 8, 8, 320)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 320)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │        10,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 80,074 (312.79 KB)

 Trainable params: 78,922 (308.29 KB)

 Non-trainable params: 1,152 (4.50 KB)

In [19]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

In [33]:
history = model.fit(
    train_data,
    epochs=50,
    #class_weight=class_weights,
    validation_data=test_data,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    ]
)

Epoch 1/50


/home/owly/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.6676 - loss: 0.6357 - val_accuracy: 0.6881 - val_loss: 0.6819
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.7206 - loss: 0.5945 - val_accuracy: 0.6881 - val_loss: 0.6806
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.7401 - loss: 0.5736 - val_accuracy: 0.6881 - val_loss: 0.6770
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.7106 - loss: 0.5961 - val_accuracy: 0.6881 - val_loss: 0.6663
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.7184 - loss: 0.5744 - val_accuracy: 0.6881 - val_loss: 0.6567
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.7080 - loss: 0.5891 - val_accuracy: 0.6881 - val_loss: 0.6530
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - accuracy: 0.7181 - loss: 0.5781 - val_accuracy: 0.6881 - val_loss: 0.6485
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.7183 - loss: 0.5846 - val_accuracy: 0.6881 - val

In [34]:
train_data.classes
np.bincount(train_data.classes)

array([1086,  435])

In [35]:
model.save("binary_mobilenet_v1.h5")